In [14]:
RADIUS = 2
NOISE = 2.5

IN_FOLDER = "/home/jovyan/data/in/"
GT_FOLDER = "/home/jovyan/data/gt/"
OUT_FOLDER = "/home/jovyan/data/out/"

!mkdir -p $OUT_FOLDER

In [15]:
!ls /home/jovyan/data

gt  in	out


In [ ]:
!python /app/wrapper.py  --ij_radius $RADIUS --ij_noise $NOISE --infolder $IN_FOLDER --gtfolder $GT_FOLDER --outfolder $OUT_FOLDER --nodownload --noexport --nometrics